## Introducción

Este ejemplo permite consultar las obras de un autor a partir del servicio de SPARQL de Wikidata y la Biblioteca Virtual Miguel de Cervantes.

In [8]:
import spacy
import requests
from SPARQLWrapper import SPARQLWrapper, JSON
!python -m spacy download es_core_news_md
 
nlp = spacy.load("es_core_news_md")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 18.5 MB/s eta 0:00:00m eta 0:00:010:01:01
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_md')


## Función que realiza la llamada al api

Utilizamos una sentencia federada que nos permite recuperar las obras de un autor a través de Wikidata.

In [9]:
def get_works(writer_name):
    
    sparql = SPARQLWrapper(
        "https://query.wikidata.org/sparql"
    )
    sparql.setReturnFormat(JSON)
    
    
    query = """
    select ?workLabel
    where {{ ?s wdt:P2799 ?id . 
       ?s wdt:P1559 ?name . 
       filter regex(?name, "{0}", "i")
       BIND(uri(concat("https://data.cervantesvirtual.com/person/", ?id)) as ?bvmcID) 
       SERVICE <http://data.cervantesvirtual.com/openrdf-sesame/repositories/data> {{
          ?bvmcID <http://rdaregistry.info/Elements/a/authorOf> ?work .
          ?work rdfs:label ?workLabel
       }}
    }} limit 10
    """.format(writer_name)
    
    #print(query)
    
    sparql.setQuery(query)
    
    try:
        ret = sparql.queryAndConvert()
        
        works = ""
        for r in ret["results"]["bindings"]:
            #print('Título: ', r['workLabel']['value'])
            works += r['workLabel']['value'] + " \n"
        return works
    except Exception as e:
        print(e)
        return None

## Definimos el patrón para solicitar las obras

Algunos ejemplos podrían ser:

- Ejemplos de obras de Miguel de Cervantes
- Ejemplos de obras de Lope de Vega

In [10]:
intencion = nlp("Ejemplos de obras de autor")

print(intencion.similarity(nlp("Ejemplos de obras de Miguel de Cervantes")))
print(intencion.similarity(nlp("Ejemplos de obras de Lope de Vega")))

0.9607676012764527
0.9428416385211742


## Función que comprueba el tiempo

In [11]:
def chatbot(query):
    statement = nlp(query)
    min_similarity = 0.75
 
    if intencion.similarity(statement) >= min_similarity:
     
        for ent in statement.ents:
            #print(ent.label_)
            if ent.label_ == "PER": 
                writer = ent.text
         
                works = get_works(writer)
                if works is not None:
                    return "Obras del escritor " + writer +": " + works
                else:
                    return "Algo no ha funcionado correctamente."
            else:
                return "Introduce un nombre de escritor."
       
    else:
        return "Lo siento, no he entendido su pregunta. Por favor, vuelva a introducir otra pregunta."

## Iniciamos el chatbot 

Ejecutamos tantas veces como queramos el chatbot.

In [12]:
print("Hola! Soy el chatbot de escritores.........")
statement = input("Cómo puedo ayudarte ?\n")
 
response = chatbot(statement)
print(response)

Hola! Soy el chatbot de escritores.........
Cómo puedo ayudarte ?
Obras de Lope de Vega
Obras del escritor Lope de Vega: Arcadia,  : prosas, y versos 
Las Serranas de la Vera 
Papeles varios 
La conquista de Mexico 
La batalla del honor 
La Conquista de Mexico : comedia famosa 
La conquista de México 
Coleccion de las obras sueltas : assi en prosa como en verso 
El Serafín humano : Comedia famosa 
Las mocedades de Bernardo del Carpio : Comedia Famosa 



## Referencias

https://github.com/hibernator11/hdh-cafe-con-2023
https://www.askpython.com/python/examples/chatbot-in-python-using-spacy